# **Pipelines**

## 1. Создание Pipeline с использованием стандартных функций 

#### Демонстрацию работы с конвеерами данных Pipeline будем демонстрировать на классификации набора данных "fetch_20newsgroups", содержащий новостные документы, разделённые по группам.  https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

In [1]:
#подключаем необходимые библиотеки
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#### Выгружаем данные, при этом перемешивая и удаляя разметочную информацию

In [24]:
data_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=228, remove=('headers', 'footers', 'quotes'))
data_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=228, remove=('headers', 'footers', 'quotes'))

In [25]:
data_train.target_names

In [26]:
# Определяем первый наш конвеер, с использованием векторизатора
preprocessor = Pipeline(steps=[('embeddings', TfidfVectorizer())])

#### Обучаем логистическую регрессию с использованием простого Pipeline

In [27]:
pipeline = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))
pipeline.fit(data_train.data, data_train.target)
pipeline.score(data_test.data, data_test.target)

#### Результат классификации - **0.67**, запомним его для финального сравнения с более сложными конвеерами

## 2. Применение функции make_pipeline с нормализатором данных

#### Примененим к данным функции нормализации, а также уменьшение размерности, для того чтобы ускорить процесс обучения логистической регрессии.

In [135]:
data_train = fetch_20newsgroups(subset='train', 
                                shuffle=True, random_state=228,
                                remove=('headers', 'footers', 'quotes'))
n_components = 2
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', use_idf=True)
X_train = vectorizer.fit_transform(data_train.data)

svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_train = lsa.fit_transform(X_train)

In [139]:
data_test = fetch_20newsgroups(subset='test', 
                               shuffle=True, random_state=228,
                               remove=('headers', 'footers', 'quotes'))

target_names = data_train.target_names

y_train, y_test = data_train.target, data_test.target
X_test = vectorizer.transform(data_test.data)
X_test = lsa.fit_transform(X_test)

In [140]:
# Обучим также логистическую регрессию
lr_clf = LogisticRegression(max_iter=10000)
lr_clf.fit(X_train, y_train)

In [141]:
lr_pred = lr_clf.predict(X_train)
train_score = accuracy_score(y_train, lr_pred) * 100
print(f"Оценка по тренировочным данным: {train_score:.2f}%")

lr_pred = lr_clf.predict(X_test)
test_score = accuracy_score(y_test, lr_pred) * 100
print(f"Оценка по тестовым данным: {test_score:.2f}%")

#### Как можно заметить результат стал хуже, потому что на него сильно влияет размерность выборки, посмотрим на результат без его применения

In [142]:
X_train = vectorizer.fit_transform(data_train.data)
lsa = make_pipeline(normalizer)
X_train = lsa.fit_transform(X_train)
X_test = vectorizer.transform(data_test.data)
X_test = lsa.fit_transform(X_test)

lr_clf = LogisticRegression(max_iter=10000)
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_train)

train_score = accuracy_score(y_train, lr_pred) * 100
print(f"Оценка по тренировочным данным: {train_score:.2f}%")

lr_pred = lr_clf.predict(X_test)
test_score = accuracy_score(y_test, lr_pred) * 100
print(f"Оценка по тестовым данным: {test_score:.2f}%")

#### Точность значительно выросла, и немного выросла по сравнению с первой классификацией.

#### Рассмотрим точность по всем группам

In [143]:
pd.DataFrame(classification_report(y_test, lr_pred, output_dict=True)).T

#### Построим матрицу корреляции для визуализации полученных данных

In [145]:
import matplotlib as plt

cm = confusion_matrix(y_test, lr_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=data_train.target_names)


fig, ax = plt.subplots(figsize=(10, 10))
disp = disp.plot(xticks_rotation='vertical', ax=ax, cmap='summer')

plt.show()

## 3. Продвинутые контейнеры данных с применением NLTK

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train = np.array(newsgroups_train.data)
y_train = np.array(newsgroups_train.target)
X_test = np.array(newsgroups_test.data)
y_test = np.array(newsgroups_test.target)

#### Рассмотрим разделение количество новостных документов по всем анализируемым группам

In [3]:
def conta_labels(y_train, y_test):
    y_train_classes = pd.DataFrame([newsgroups_train.target_names[i] for i in newsgroups_train.target])[0]
    y_test_classes = pd.DataFrame([newsgroups_test.target_names[i] for i in newsgroups_test.target])[0]
    
    contagem_df = pd.concat([y_train_classes.value_counts(),
                             y_test_classes.value_counts()],
                            axis=1, 
                            keys=["Тренировочные", "Тестовые"], 
                            sort=False)
    
    contagem_df["Общие"] = contagem_df.sum(axis=1)
    contagem_df.loc["Сумма"] = contagem_df.sum(axis=0)
    
    return contagem_df

newsgroups_df_labels = conta_labels(y_train, y_test)
newsgroups_df_labels

In [4]:
newsgroups_df_labels.iloc[:-1,:-1].plot.barh(stacked=True, 
                                    figsize=(10, 8),
                                    color = 'cym',
                                    title="Количество документов на каждый класс");

#### Используем модуль для обработки языка - NLTK. И создадим класс, который получает на вход новостной документ, а возравращает наборы слов с удалёнными стоп-словами и пунктуационными знаками.

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics

In [6]:
import string
import re
import nltk

class NLTKTokenizer(): 
    def __init__(self):
        self.lemmatizer = nltk.stem.WordNetLemmatizer()
        self.stopwords = nltk.corpus.stopwords.words('english')
        self.english_words = set(nltk.corpus.words.words())
        self.pontuacao = string.punctuation

    def __call__(self, doc):
        doc = doc.lower()       
        doc = re.sub(r'[0-9]+', 'num', doc)
        doc = re.sub(r'[_]+', 'underline', doc)
        doc = re.sub(r'(http|https)://[^\s]*', 'http', doc)
        doc = re.sub(r'[^\s]+@[^\s]+', 'email', doc) 
        doc = re.sub(r'\\r\\n', ' ', doc)
        doc = re.sub(r'\W', ' ', doc)
        doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc)
        doc = re.sub(r'\^[a-zA-Z]\s+', ' ', doc) 
        doc = re.sub(r'\s+', ' ', doc, flags=re.I)
        palavras = []
        for word in nltk.word_tokenize(doc):
            if word in self.stopwords:
                continue
            if word in self.pontuacao:
                continue
            if word not in self.english_words:
                continue
            
            word = self.lemmatizer.lemmatize(word)
            palavras.append(word)
        
        return palavras

#### Преобразуем данные в векторные признаки с помощью библиотек NLTK и регулярных выражений.


In [9]:
vectorizator = CountVectorizer()
v1 = vectorizator.fit_transform(X_train)

features = vectorizator.get_feature_names()
v1_df = pd.DataFrame(v1.toarray(), columns = features)
v1_df

### Добавляем название признакам и используем наш токенизатор. (Лемматизация, удаление стоп-слов и неизвестных слов)

In [10]:
nltk_vectorizator = CountVectorizer(tokenizer=NLTKTokenizer())
v2 = nltk_vectorizator.fit_transform(X_train)

features = nltk_vectorizator.get_feature_names()
v2_df = pd.DataFrame(v2.toarray(), columns = features)
v2_df

#### Логистическая регрессия: с применением конвейера данных (TfidfVectorizer, TfidfTransformer, LogisticRegression)

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

text_clf_logistic_regression = Pipeline([('vect', TfidfVectorizer(max_df=0.5, min_df=2, 
                                                              stop_words='english', 
                                                              use_idf=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(penalty='l2', 
                                                dual=False, 
                                                tol=0.001, 
                                                C=1.0, 
                                                fit_intercept=True, 
                                                intercept_scaling=1, 
                                                class_weight=None, 
                                                random_state=None, 
                                                solver='lbfgs', 
                                                max_iter=1000, 
                                                multi_class='multinomial', 
                                                verbose=0, 
                                                warm_start=False, 
                                                n_jobs=None, 
                                                l1_ratio=None)),
                     ])

text_clf_logistic_regression.fit(X_train, y_train)
predicted = text_clf_logistic_regression.predict(X_test)
lr_tf_score = accuracy_score(y_test, predicted)
print(metrics.classification_report(y_test, predicted))

#### Логистическая регрессия: с применением конвейера данных (NLTKTokenizer, TfidfTransformer, LogisticRegression)

In [43]:
text_clf_logistic_regression = Pipeline([('vect', CountVectorizer(tokenizer=NLTKTokenizer())),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(penalty='l2', 
                                                dual=False, 
                                                tol=0.001, 
                                                C=1.0, 
                                                fit_intercept=True, 
                                                intercept_scaling=1, 
                                                class_weight=None, 
                                                random_state=None, 
                                                solver='lbfgs', 
                                                max_iter=1000, 
                                                multi_class='multinomial', 
                                                verbose=0, 
                                                warm_start=False, 
                                                n_jobs=None, 
                                                l1_ratio=None)),
                     ])

text_clf_logistic_regression.fit(X_train, y_train)
predicted = text_clf_logistic_regression.predict(X_test)
lr_nltk_score = accuracy_score(y_test, predicted)
print(metrics.classification_report(y_test, predicted))

#### Классификация ближайших соседей

In [44]:
from sklearn.neighbors import KNeighborsClassifier

text_clf_knn = Pipeline([('vect', TfidfVectorizer(max_df=0.5, min_df=2, 
                                                              stop_words='english', 
                                                              use_idf=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier(n_neighbors=5, 
                                                  weights='uniform', 
                                                  algorithm='auto', 
                                                  leaf_size=30, 
                                                  p=2, 
                                                  metric='minkowski', 
                                                  metric_params=None, 
                                                  n_jobs=None)),
                     ])

text_clf_knn.fit(X_train, y_train)
predicted = text_clf_knn.predict(X_test)
knn_tf_score = accuracy_score(y_test, predicted)
print(metrics.classification_report(y_test, predicted))

In [45]:
text_clf_knn = Pipeline([('vect', CountVectorizer(tokenizer=NLTKTokenizer())),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier(n_neighbors=5, 
                                                  weights='uniform', 
                                                  algorithm='auto', 
                                                  leaf_size=30, 
                                                  p=2, 
                                                  metric='minkowski', 
                                                  metric_params=None, 
                                                  n_jobs=None)),
                     ])

text_clf_knn.fit(X_train, y_train)
predicted = text_clf_knn.predict(X_test)
knn_nltk_score = accuracy_score(y_test, predicted)
print(metrics.classification_report(y_test, predicted))

#### Классификация случайного леса

In [52]:
text_clf_rf = Pipeline([('vect', TfidfVectorizer(max_df=0.5, min_df=2, 
                                                              stop_words='english', 
                                                              use_idf=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=100, 
                                                    criterion='gini', 
                                                    max_depth=None, 
                                                    min_samples_split=2, 
                                                    min_samples_leaf=1, 
                                                    min_weight_fraction_leaf=0.0, 
                                                    max_features='auto', 
                                                    max_leaf_nodes=None, 
                                                    min_impurity_decrease=0.0, 
                                                    bootstrap=True, 
                                                    oob_score=False, 
                                                    n_jobs=None, 
                                                    random_state=None, 
                                                    verbose=0, 
                                                    warm_start=False))])

text_clf_rf.fit(X_train, y_train)
predicted = text_clf_rf.predict(X_test)
rfc_tf_score = accuracy_score(y_test, predicted)
print(metrics.classification_report(y_test, predicted))

In [53]:
text_clf_rf = Pipeline([('vect', TfidfVectorizer(max_df=0.5, min_df=2, 
                                                              stop_words='english', 
                                                              use_idf=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=100, 
                                                    criterion='gini', 
                                                    max_depth=None, 
                                                    min_samples_split=2, 
                                                    min_samples_leaf=1, 
                                                    min_weight_fraction_leaf=0.0, 
                                                    max_features='auto', 
                                                    max_leaf_nodes=None, 
                                                    min_impurity_decrease=0.0, 
                                                    bootstrap=True, 
                                                    oob_score=False, 
                                                    n_jobs=None, 
                                                    random_state=None, 
                                                    verbose=0, 
                                                    warm_start=False))])

text_clf_rf.fit(X_train, y_train)
predicted = text_clf_rf.predict(X_test)
rfc_nltk_score = accuracy_score(y_test, predicted)
print(metrics.classification_report(y_test, predicted))

#### Анализ полученных результатов

In [54]:
models = [('LogisticRegression_TV', lr_tf_score),
          ('LogisticRegression_NLTK', lr_nltk_score),
          ('KNeighborsClassifier_TV', knn_tf_score),
          ('KNeighborsClassifier_NLTK', knn_nltk_score),
          ('RandomForestClassifier_TV', rfc_tf_score),
          ('RandomForestClassifier_NLTK', rfc_nltk_score)
          ]

In [85]:
total = pd.DataFrame(data=models, columns=['Model', 'Score'])
total = total.sort_values(by=['Score'], ignore_index=True, ascending=False)
total

#### Наилучший классификатор оказался - Логистическая регрессия, где мы снова смогли улучшить результат.

#### В каждом из классификаторов мы применяли 2 типа векторизаторов - собранный нами выше с примением модуля NLTK и TfidfTransformer, с немного изменными параметрами. Pipeline дают нам возможности удобно работать с данными и применять к ним одновременно несколько необходимых инструментов. Это позволяет нам последовательно обработать данные, без их утечки или потери.

#### Подводя итоги анализа данных с применением контейнера данных Pipeline:
+ Наилучший результат показала модель Логистической регрессии с TF-IDF векторизатором.
+ Векторизатор на основе предобработки данных NLTK показал результат хуже, только на одной модели он достиг лучшей точности - KNeighborsClassifier.
+ Применение классификации ближайших соседий было бесполезно, что и стоило доказать, так как наши данные находятся в перемешанном варианте.
